In [ ]:
import time
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *
from scipy.io import wavfile
import torch
import numpy as np
from model_logging import TensorboardLogger
import torch.nn.functional as F
from tqdm import tqdm
import os

# Debug printing function
debug_enabled = False
def debug_print(msg):
    if debug_enabled:
        print(msg)

def set_debug(enabled):
    global debug_enabled
    debug_enabled = enabled

# Set debug printing off by default
set_debug(False)

def get_device():
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    return "cpu"

# Check device availability
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"MPS built: {torch.backends.mps.is_built()}")
print(f"CUDA available: {torch.cuda.is_available()}")

device = get_device()
print(f"Using device: {device}")

In [11]:
# Memory dataset for better performance
class MemoryDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.length = len(dataset)
        self.train = dataset.train if hasattr(dataset, 'train') else True

        # Print sample item shape during initialization
        sample_item = self.dataset[0]
        print("\nDataset sample info:")
        print(f"Sample item type: {type(sample_item)}")
        if isinstance(sample_item, tuple):
            print(f"Sample x shape: {sample_item[0].shape}")
            print(f"Sample target shape: {sample_item[1].shape}")
        else:
            print(f"Sample shape: {sample_item.shape}")

    def __getitem__(self, idx):
        item = self.dataset[idx]
        # Print shape occasionally for debugging
        if idx % 1000 == 0 and debug_enabled:
            if isinstance(item, tuple):
                print(f"\nBatch {idx} shapes:")
                print(f"x: {item[0].shape}")
                print(f"target: {item[1].shape}")
        return item

    def __len__(self):
        return self.length

In [ ]:
model = WaveNetModel(layers=6,
                     blocks=4,
                     dilation_channels=16,
                     residual_channels=16,
                     skip_channels=32,
                     output_length=8,
                     bias=False)

# Load latest model if available
model = load_latest_model_from('snapshots', use_cuda=False)

# Ensure model is on correct device
model = model.to(device)
print(f"Model device after transfer: {next(model.parameters()).device}")
print(f"Start conv weight device: {model.start_conv.weight.device}")

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

In [ ]:
# Create dataset
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=20)

# Load dataset file and print info
print("Loading dataset file:", data.dataset_file)
with np.load(data.dataset_file) as dataset:
    print("Available keys in dataset:", dataset.files)
    data.data = dataset['arr_0']

print('the dataset has ' + str(len(data)) + ' items')
print(f"Dataset type: {type(data.data)}")
print(f"Dataset shape: {data.data.shape}")
print(f"Dataset dtype: {data.data.dtype}")

In [ ]:
# Create memory dataset for better performance
memory_dataset = MemoryDataset(data)

# Create trainer with memory_dataset
trainer = WavenetTrainer(
    model=model,
    dataset=memory_dataset,
    batch_size=32, # JOS: was 8, then 16
    val_batch_size=32, # JOS: was 32
    val_subset_size=1000,  # Increased from 500
    lr=0.0005,  # Reduced from 0.001
    weight_decay=0.01,  # Added L2 regularization
    gradient_clipping=1,
    snapshot_interval=500,  # was 1000
    snapshot_path='snapshots',
    val_interval=1000,
    num_workers=4
)

In [ ]:
def generate_and_log_samples(model, step, temperature=1.0):
    """
    Generate audio samples and prepare them for logging.
    Args:
        model: The trained WaveNet model
        step: Current training step (for logging)
        temperature: Controls randomness in generation
    Returns:
        Dictionary containing the generated audio data
    """
    # Generate samples
    samples = generate_audio(model, length=16000, temperature=temperature)
    
    # Create a dictionary with the samples
    data = {
        'temperature': temperature,
        'samples': samples,
        'step': step,
        'sample_rate': 16000
    }
    
    return data

# Set up TensorboardLogger
try:
    logger = TensorboardLogger(log_interval=200,
                            validation_interval=200,
                            generate_interval=500,
                            generate_function=generate_and_log_samples,
                            log_dir="logs")
    print("TensorboardLogger initialized successfully")
except Exception as e:
    print(f"Error initializing TensorboardLogger: {e}")
    print("Continuing without logging...")

In [16]:
# Updated generate_audio function
def generate_audio(model, length=16000, temperature=1.0):
    """
    Generate audio samples using the trained model.
    Args:
        model: Trained WaveNet model
        length: Number of samples to generate
        temperature: Controls randomness (higher = more random, lower = more deterministic)
    """
    model.eval()  # Set to evaluation mode
    
    # Move model to device if not already
    model = model.to(device)

    # Start with zeros and convert to one-hot encoding
    current_sample = torch.zeros(1, 256, model.receptive_field).to(device)
    generated_samples = []

    print(f"\nGenerating {length} samples...")

    with torch.no_grad():
        # We need to generate samples in chunks since the model outputs multiple samples at once
        for i in tqdm(range(0, length, model.output_length)):
            # Get model prediction
            output = model(current_sample)
            
            # Reshape output to [batch_size, output_length, num_classes]
            output = output.view(1, model.output_length, 256)
            
            # Apply temperature
            if temperature != 1:
                output = output / temperature

            # Process each output step
            for j in range(min(model.output_length, length - i)):
                # Get probabilities for current timestep
                probabilities = F.softmax(output[0, j], dim=0)
                
                # Sample from the output distribution
                next_sample_idx = torch.multinomial(probabilities, 1).item()
                
                # Append to generated samples
                generated_samples.append(next_sample_idx)
                
                # Create one-hot encoding for the new sample
                if j < model.output_length - 1 or i + model.output_length < length:
                    next_sample_onehot = torch.zeros(1, 256, 1).to(device)
                    next_sample_onehot[0, next_sample_idx, 0] = 1
                    
                    # Shift input window and add new sample
                    current_sample = torch.roll(current_sample, -1, dims=2)
                    current_sample[:, :, -1] = next_sample_onehot.squeeze(-1)

    # Convert to numpy array
    samples = np.array(generated_samples, dtype=np.int16)

    # Scale back to audio range
    samples = samples - 128

    return samples

In [ ]:
# Set up TensorboardLogger
try:
    logger = TensorboardLogger(log_interval=200,
                            validation_interval=200,
                            generate_interval=500,
                            generate_function=generate_and_log_samples,
                            log_dir="logs")
    print("TensorboardLogger initialized successfully")
except Exception as e:
    print(f"Error initializing TensorboardLogger: {e}")
    print("Continuing without logging...")

In [ ]:
# Start training with error handling
print('\nStarting training...')
tic = time.time()
try:
    trainer.train(epochs=8) # JOS: was 20
    # print("TRAINING DISABLED")
    pass
except Exception as e:
    print(f"\nError during training:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    print(f"Model device: {next(model.parameters()).device}")
    raise
toc = time.time()
print('Training took {} seconds.'.format(toc - tic))

In [ ]:
toc = time.time()
print('Training took more than {} seconds.'.format(toc - tic))

In [ ]:
# Set to evaluation mode
memory_dataset.train = False
trainer.dataloader.dataset.train = False

print("Dataloader length:", len(trainer.dataloader))
print("Test dataset length:", len(memory_dataset))
print("Sample length:", data.item_length)

In [ ]:
# Generate audio samples
# Get a starting sample from the dataset
start_data = data[100][0]
start_data = torch.max(start_data, 0)[1]
print("Starting data shape:", start_data.shape)
print("Starting data sample:", start_data)

In [ ]:
# Progress callback for generation
def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

model = load_latest_model_from('snapshots')
model = model.to(device)

# Get a starting sample from the dataset
start_data = data[100][0]
start_data = torch.max(start_data, 0)[1]
print("Starting data shape:", start_data.shape)
print("Starting data sample:", start_data)

# Make sure model is in evaluation mode
model.eval()

# Use the simpler generate_audio function instead of generate_fast
print("\nGenerating audio samples...")
generated1 = generate_audio(model, length=160000, temperature=1.0)


In [ ]:
# Play the generated audio
import IPython.display as ipd

ipd.Audio(generated1, rate=16000)

In [ ]:
# Save the generated audio
output_file = "generated_audio.wav"
wavfile.write(output_file, 16000, generated1)
print(f"Generated audio saved to: {output_file}")

In [ ]:
# Visualize the generated audio
%matplotlib inline
from matplotlib import pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(generated1); ax1.set_title('Raw audio signal')
ax2.specgram(generated1, Fs=16000); ax2.set_title('Spectrogram');